In [1]:
import cv2
import mediapipe as mp
import torch 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [17]:
# For static images:
with mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.5) as pose:
  for idx, file in enumerate(file_list):
    image = cv2.imread(file)
    image_height, image_width, _ = image.shape
    # Convert the BGR image to RGB before processing.
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if not results.pose_landmarks:
      continue
    print(
        f'Nose coordinates: ('
        f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width}, '
        f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y * image_height})'
    )
    # Draw pose landmarks on the image.
    annotated_image = image.copy()
    # Use mp_pose.UPPER_BODY_POSE_CONNECTIONS for drawing below when
    # upper_body_only is set to True.
    mp_drawing.draw_landmarks(
        annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imshow('MediaPipe Pose', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

NameError: name 'file_list' is not defined

In [ ]:
# 모델(yolov5) 가져오기
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [11]:
# 우리는 사람을 구분하는 것에 관심이 있으니깐
yolo_model.classes=[0]

In [13]:
# global variables
pose_estimator = []
pose_estimator_dim = []
# For each object detected
# WHICH POSE ESTIMATOR TO USE.
selected_pose_idx=0
                                                     
if(len(pose_estimator)==0): 
   pose = mp_pose.Pose(min_detection_confidence=0.6,     
             min_tracking_confidence=0.6)
   pose_estimator.append(pose)    
   pose_estimator_dim.append(cord_thres)
                             
   selected_pose_idx = len(pose_estimator)-1                            
elif(labels >len(pose_estimator)):
   thresholdForNew = 100
   prev_high_score = 0
   selected_pose_idx_high =0
   prev_low_score = 1000000000
   selected_pose_idx_low =0
   pose_idx = 0
   for dim in pose_estimator_dim:
      score = compareDist(dim,cord_thres)
      if(score > prev_high_score):
         selected_pose_idx_high  =  pose_idx
         prev_high_score = score
      if(score < prev_low_score):                                        
         selected_pose_idx_low  =  pose_idx
         prev_low_score = score
      pose_idx+=1
   if prev_high_score > thresholdForNew:
      pose = mp_pose.Pose(min_detection_confidence=0.6,min_tracking_confidence=0.6)
      pose_estimator.append(pose)    
      pose_estimator_dim.append(cord_thres)
      selected_pose_idx = len(pose_estimator)-1 
   else:
      selected_pose_idx = selected_pose_idx_low
   pose_estimator_dim[selected_pose_idx]= cord_thres
                                    
else:
   pose_idx = 0
   prev_score = 1000000000                                
   for dim in pose_estimator_dim:
      score = compareDist(dim,[x_min, y_min, box_width, box_height])
      if(score < prev_score):                                        
         selected_pose_idx  =  pose_idx
         prev_score = score   
      pose_idx+=1
   pose_estimator_dim[selected_pose_idx]= cord_thres

In [ ]:
# 카메라 연결. 맥에서는 숫자 2로 하는게 낫대
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture(video_path)

# 웹캠 프레임 단위로 돌린다
while cap.isOpened():
    ret, frame = cap.read()
        
    # Recolor Feed 
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
        
    # Make Detections
    result = yolo_model(image)
    # print(results.face_Landmarks)
    # face_landmarks, pose_landmarks, Left_hand_landmarks, right_hand_landmarks   
    # Recolor image back to BGR for rendering
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    img_list = []
    
    MARGIN = 10
    
    for (xmin, ymin, xmax, ymax, confidence, clas) in result.xyxy[0].tolist():
        # Initiate holistic model
      with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
        # 이게 사람 구분하는 코드
        #Media pose prediction ,we are 
        results = holistic.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:])

        #Draw landmarks on image, if this thing is confusing please consider going through numpy array slicing 
        # 2. Right hand
        mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )
        # 3. Left hand
        mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )
        img_list.append(image[int(ymin):int(ymax),int(xmin):int(xmax):])
        
        # Happy를 알려주는 데이터 추출(우리가 웃는 표정을 데이터로 뽑는 것)
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Concate rows
            row = pose_row

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            cv2.rectangle(image, 
                          (coords[0], coords[1] + 5), 
                          (coords[0] + len(body_language_class) * 20, coords[1] - 30),
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords,
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            """
            #Get status box
            # 첫 번째 인수: 시각형 시작 좌표, 두 번째: 사각형 크기
            cv2.rectangle(image, (0, 0), (300, 60), (245, 117, 16), -1)
            
            
            # Display Class
            cv2.putText(image, 'CLASS'
                       , (125, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                       , (120, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
           
            
            # Display Probability
            cv2.putText(image, 'PROB'
                       , (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)], 2))
                       , (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            """
        except:
            pass

    cv2.imshow('Raw Webcam Feed', image)
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
            
cap.release()
cv2.destroyAllWindows()